# Trabajo Final procesamiento de lenguaje natural (NLP) <a class="tocSkip">
## Universidad Pontificia Bolivariana <a class="tocSkip">

**Estudiante:** David Alejandro Arroyave

**ID:** 000573992



**Subir Este notebook de Trabajo Final en su repositorio de GitHub. y enviar el link en **Microsoft TEAMS**

Docente: [Jose R. Zapata](https://joserzapata.github.io)
- https://joserzapata.github.io
- https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/       


## Objetivo del Trabajo
Realizar un proceso de Procesamiento de lenguaje natural (NLP) para dejar los datos preparados para ser usados con algoritmos de Machine Learning para Clasificación como objetivo final del trabajo.

los datos preparados para ser usados con algoritmos de Machine Learning para Regresión o Clasificación como objetivo final del trabajo.

El trabajo se realizara en este jupyter notebook y subirlo a su repositorio de github creado en clase. (**Recuerde poner su nombre e información**)

## Las actividades a realizar
    

1) Limpiar los datos de texto.

   - https://joserzapata.github.io/courses/nlp/procesamiento-basico/

   - https://joserzapata.github.io/courses/nlp/preprocesamiento-texto/

2) Realizar la representación de texto:

   - https://joserzapata.github.io/courses/nlp/representaciones/

   - Tokenización
   - Lematización o stemming
   - Representación de los datos de texto (Bag of Words o TF-IDF)

3) Utilice un modelo de Machine Learning para clasificación para entrenar y evaluar el modelo.
        
    - Calcule estas las métricas de evaluación (accuracy, precision, recall, f1-score) - https://joserzapata.github.io/courses/python-ciencia-datos/clasificacion/#evaluacion-modelo-simple


    
   
    
*NOTA: No dude en contactarme para cualquier pregunta o inquietud :) por el chat de Teams o al correo
joser.zapata@upb.edu.co*

## EVALUACIÓN


|Porcentaje en la evaluación | Descripción| Nada | Incompleto | Completo 
| :---: |:---: |:---: |:---: |:---: 
| 5 % |**limpieza de los datos** |
| 5 % | **Representacion de los datos** |
| 5 % | **Machine Learning** <br> Entrenar y evaluar el modelo propuesto |


In [1]:
import pandas as pd
import pyarrow 
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)
from sklearn.model_selection import train_test_split

Carga del dataset

El archivo  se carga en aproximadamente 10 segundos

In [2]:
# Usar su ID UPB Ejemplo: "0028984798"
id_upb = "000573992"
data_reviews = pd.read_parquet("https://www.dropbox.com/scl/fi/gvk9yj8cn96oocr9z058x/filmaffinity_reviews_dataset.parquet?rlkey=xgvr00zvkxbkwqqavqutpsshg&st=xjb7xze9&dl=1")
data_reviews = data_reviews.sample(n=50_000, random_state=int(id_upb))
data_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 128307 to 106374
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   author_review_desc  50000 non-null  string
 1   author_rating       50000 non-null  int64 
dtypes: int64(1), string(1)
memory usage: 1.1 MB


Ejemplo de algunas filas del dataset

In [3]:
data_reviews.sample(5)

,author_review_desc,author_rating
130027,"\nPasable película de Ventura Pons, que se sit...",5
37774,\n[1] Peculiar ‘love story’ en la Holanda de l...,5
128071,"\nPara entender ""Distrito 9"" hay que conocer l...",8
48692,\nFrívola y divertida!!!\n\nSé que es una come...,6
117002,"\nLos 39 escalones tiran hacia arriba, de ahí ...",6


Evaluar los valores nulos

In [4]:
data_reviews.isnull().sum()

author_review_desc    0
author_rating         0
dtype: int64

En este caso no hay nulos.

Revisar si hay valores duplicados:

In [5]:
data_reviews.duplicated().sum()

np.int64(1)

No hay  valores duplicados.

## Clasificación Tradicional para Análisis de Sentimientos y Categorías 👍 👎



Definición: Se considera reseña positiva cuando la puntuación ("author_rating") es mayor que 6; negativa en caso contrario.

Crear la variable binaria de sentimiento: 1 (positivo) si author_rating > 6, 0 (negativo) de lo contrario

In [6]:
UMBRAL = 6
data_reviews["sentiment_bin"] = (data_reviews["author_rating"] > UMBRAL).astype(int)

In [7]:
# Dividir los datos en conjuntos de entrenamiento y prueba

X_data = data_reviews["author_review_desc"]
y_data = data_reviews["sentiment_bin"]


Dividir el dataset en entrenamiento y prueba

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data,
    y_data,
    test_size=0.2,
    stratify=y_data,  # Mantener la proporción de clases en ambos conjuntos
    random_state=42,
)

## Análisis exploratorio de los datos

para determinar que tipo de limpieza se debe realizar a los datos de texto de `X_train`

## 1) Limpieza de los datos de texto

Tomar los datos de `X_train` y aplicar las funciones de limpieza que considere necesarias

Ayudas:

- Convertir a minúsculas
- Eliminar caracteres especiales y números
- Corregir palabras mal escritas
- etc


In [9]:
import re

def limpiar_texto(texto):
    """
    Función para limpiar el texto de las reseñas:
    1. Convierte a minúsculas.
    2. Elimina puntuación, números y caracteres especiales.
    Se preservan caracteres del español (á, é, í, ó, ú, ü, ñ).
    3. Elimina espacios en blanco extra.
    """
    if not isinstance(texto, str):
        return ""

    texto = texto.lower()
    texto = re.sub(r"[^a-záéíóúüñ\s]", "", texto)
    texto = re.sub(r"\s+", " ", texto).strip()

    return texto


print(f"Limpiando {len(X_train)} reseñas de X_train...")
X_train_limpio = X_train.apply(limpiar_texto)

print(f"Limpiando {len(X_test)} reseñas de X_test...")
X_test_limpio = X_test.apply(limpiar_texto)

print("\n--- Limpieza completada ---")

print("\nEJEMPLO DE LIMPIEZA:")
print("------------------------")
print("RESEÑA ORIGINAL (X_train):")
print(X_train.iloc[0])
print("------------------------")
print("RESEÑA LIMPIA (X_train_limpio):")
print(X_train_limpio.iloc[0])




Limpiando 40000 reseñas de X_train...


Limpiando 10000 reseñas de X_test...

--- Limpieza completada ---

EJEMPLO DE LIMPIEZA:
------------------------
RESEÑA ORIGINAL (X_train):

Es a fin de cuentas, una película maniqueísta y tendenciosa, que hace ver que en la Guerra civil española había buenos y malos, cuando esto es totalmente falso. Lo que no se puede hacer es poner al profesor rojo; entrañable, y a los nacionales tan malos, porque extrapolando el asunto, refleja que todos los republicanos eran buenos y todos los nacionales malos. 

En fin buena puesta en escena, pero el trasfondo tendencioso, una buena pelicula en el trato de estos temas debe ser mas objetiva y con un conocimiento mas amplio de la historia. Habrá que hacer entrega de dos gallinas a José Luis Cuerda por su lucidez en esta pelicula, como hace el nacional al intentar sobornar al profesor rojo, que risa me hizo al verlo :) no podrían poner a los nacionales con parches en los ojos. En fin.

------------------------
RESEÑA LIMPIA (X_train_limpio):
es a fin

## 2) Representacion del texto

Luego de tener los datos limpios, realizar la representación de los datos de texto para poder usarse en modelos de machine learning.

In [ ]:
# Representación Bag-of-Words con CountVectorizer#

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(X_train_limpio)
print("Dimensiones de la matriz TF-IDF:", tfidf_matrix.shape)
print(
    "Ejemplo de términos (TF-IDF):",
    np.random.choice(tfidf.get_feature_names_out(), size=10, replace=False),
)

Dimensiones de la matriz TF-IDF: (40000, 151659)
Ejemplo de términos (TF-IDF): ['explotémoslo' 'httpscineparatodaswordpresscom' 'elegirla' 'umpa'
 'rasgasen' 'legión' 'teleobjetivo' 'tráumas' 'creíblescuando' 'astra']


In [ ]:
#Representación TF-IDF con TfidfVectorizer#

In [19]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(X_train_limpio)
print("Dimensiones de la matriz TF-IDF:", tfidf_matrix.shape)
print(
    "Ejemplo de términos (TF-IDF):",
    np.random.choice(tfidf.get_feature_names_out(), size=10, replace=False),
)


Dimensiones de la matriz TF-IDF: (40000, 151659)
Ejemplo de términos (TF-IDF): ['tudeski' 'dougal' 'chyler' 'tramoya' 'keaton' 'kristen' 'espeso'
 'filosensibloides' 'colaboraba' 'riddler']


## 3) Entrenar un modelo de machine learning de clasificación

Utilizar un modelo de clasificación para entrenar y evaluar el modelo con los datos preparados.


In [ ]:
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)

# (Asumimos que las variables X_train_tfidf, y_train, X_test_tfidf, y_test
#  existen en memoria desde los pasos anteriores)

print("--- Iniciando Paso 3: Entrenamiento del Modelo ---")

# --- 1. Instanciar el Modelo ---
# Elegimos LogisticRegression. Es un modelo lineal robusto y rápido,
# ideal para datos TF-IDF que son "dispersos" (muchos ceros).
# - random_state=42: Asegura que obtengamos los mismos resultados cada vez.
# - max_iter=1000: Aumentamos las iteraciones para asegurar que el modelo
#   "converja" (encuentre la mejor solución) con nuestros 5000 features.
model = LogisticRegression(random_state=42, max_iter=1000)

print(f"Modelo seleccionado: {type(model).__name__}")

# --- 2. Entrenar el Modelo ---
print("\nEntrenando el modelo con los datos de X_train_tfidf...")
# Usamos .fit() para que el modelo aprenda la relación entre
# la matriz TF-IDF (las palabras) y el sentimiento (0 o 1).
model.fit(X_train_tfidf, y_train)

print("¡Modelo entrenado exitosamente!")

# --- 3. Evaluar el modelo con los datos de prueba ---

print("\n--- Iniciando Evaluación del Modelo ---")
print("Generando predicciones en el conjunto de prueba (X_test_tfidf)...")

# Usamos .predict() en los datos de prueba (X_test_tfidf)
# El modelo NUNCA vio estos datos durante el entrenamiento.
y_pred = model.predict(X_test_tfidf)

# Opcional: Calcular probabilidades para ROC-AUC
# (Mide qué tan bien distingue el modelo entre clases)
y_pred_proba = model.predict_proba(X_test_tfidf)[:, 1]

print("Predicciones generadas.")

# --- 4. Calcular y Reportar Métricas ---
# Comparamos las predicciones (y_pred) con las etiquetas reales (y_test)

print("\n--- Métricas de Evaluación ---")

# Accuracy: ¿Qué porcentaje de reseñas clasificó correctamente?
# (Correctas / Total)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy (Exactitud): {acc:.4f}")

# Precision: De todas las que el modelo dijo que eran "Positivas",
# ¿cuántas realmente lo eran?
# (Importante si queremos evitar "Falsos Positivos")
prec = precision_score(y_test, y_pred)
print(f"Precision: {prec:.4f}")

# Recall (Sensibilidad): De todas las reseñas que *eran* "Positivas",
# ¿cuántas logró encontrar el modelo?
# (Importante si queremos encontrar todos los "Positivos Reales")
rec = recall_score(y_test, y_pred)
print(f"Recall (Sensibilidad): {rec:.4f}")

# F1-Score: La media armónica entre Precision y Recall.
# Un buen balance general entre ambas.
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

# ROC-AUC: Mide la habilidad del modelo para discriminar
# entre la clase positiva y negativa. (1.0 es perfecto, 0.5 es azar)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC AUC Score: {roc_auc:.4f}")

# Reporte de Clasificación: Un resumen completo
print("\n--- Reporte de Clasificación Detallado ---")
# Muestra Precision, Recall y F1 para ambas clases (0 y 1)
print(
    classification_report(y_test, y_pred, target_names=["Negativo (0)", "Positivo (1)"])
)


# --- 5. Guardar el Modelo Entrenado y el Vectorizador ---
# Es crucial guardar también el 'vectorizer' del Paso 2,
# porque para predecir en nuevos datos, necesitamos aplicar
# EXACTAMENTE la misma transformación TF-IDF.

print("\n--- Guardando artefactos del modelo ---")

# Guardamos el modelo entrenado
joblib.dump(model, "modelo_sentimientos.joblib")
print("Modelo guardado en 'modelo_sentimientos.joblib'")

# (Asumiendo que 'vectorizer' es el nombre de tu TfidfVectorizer del Paso 2)
try:
    joblib.dump(vectorizer, "vectorizador_tfidf.joblib")
    print("Vectorizador guardado en 'vectorizador_tfidf.joblib'")
except NameError:
    print("ADVERTENCIA: No se pudo guardar el 'vectorizer'.")
    print("Asegúrate de que la variable 'vectorizer' (del Paso 2) esté en memoria.")
except Exception as e:
    print(f"Error al guardar el vectorizador: {e}")

print("\n--- Proceso completado ---")


--- Iniciando Paso 3: Entrenamiento del Modelo ---
Modelo seleccionado: LogisticRegression

Entrenando el modelo con los datos de X_train_tfidf...
¡Modelo entrenado exitosamente!

--- Iniciando Evaluación del Modelo ---
Generando predicciones en el conjunto de prueba (X_test_tfidf)...
Predicciones generadas.

--- Métricas de Evaluación ---
Accuracy (Exactitud): 0.8020
Precision: 0.8037
Recall (Sensibilidad): 0.8345
F1 Score: 0.8188
ROC AUC Score: 0.8837

--- Reporte de Clasificación Detallado ---
              precision    recall  f1-score   support

Negativo (0)       0.80      0.76      0.78      4639
Positivo (1)       0.80      0.83      0.82      5361

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000


--- Guardando artefactos del modelo ---
Modelo guardado en 'modelo_sentimientos.joblib'
Vectorizador guardado en 'vectorizador_tfidf.joblib'

--- Proceso completado ---


Métricas de Evaluación Clave:

Accuracy (Exactitud): 0.8020 (80.2%)

El modelo clasificó correctamente 8 de cada 10 reseñas en el conjunto de prueba. Este es un rendimiento sólido y robusto.

ROC AUC Score: 0.8837 (88.4%)

Esta métrica, que mide la capacidad del modelo para discriminar entre una reseña positiva y una negativa, es muy alta. Un valor de 0.88 (cercano a 1.0) indica que el modelo es muy confiable para distinguir las clases.

Reporte de Clasificación (F1-Score):

Positivo (1): El modelo obtuvo un F1-Score de 0.82.

Negativo (0): El modelo obtuvo un F1-Score de 0.78.

El rendimiento es balanceado (Macro Avg F1-Score: 0.80), lo que demuestra que el modelo no tiene un sesgo significativo hacia una clase y es competente identificando tanto reseñas positivas como negativas.

Conclusión Final:

El pipeline de NLP clásico (Limpieza -> Stemming -> TF-IDF) combinado con un modelo lineal (Regresión Logística) ha demostrado ser muy efectivo, alcanzando más de un 80% de exactitud. Los artefactos del modelo (modelo_sentimientos.joblib) y el vectorizador (vectorizador_tfidf.joblib) se han guardado exitosamente y están listos para ser desplega

### Evaluar el modelo con los datos de prueba

Usar el modelo para predecir en `X_test` y evaluar con `y_test`

**Nota:** Recuerde que `X_test` debe pasar por los mismos procesos de limpieza y representación que `X_train`

In [22]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

NameError: name 'y_pred' is not defined

**Guardar el modelo**

In [ ]:
import joblib

## Librerías Usadas

In [ ]:
from watermark import watermark

print(watermark(python=True, iversions=True, globals_=globals()))

---

## Referencias
- [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
- [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
- [Classification of text documents using sparse features](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html)
- Ejemplo de Entrenamiento y selección de Modelo de machine learning entre varios modelos <https://joserzapata.github.io/post/ciencia-datos-proyecto-python/6-model_selection/>
- https://joserzapata.github.io/courses/python-ciencia-datos/python/
- https://joserzapata.github.io/courses/python-ciencia-datos/pandas/
- https://joserzapata.github.io/courses/python-ciencia-datos/machine-learning/

Docente: [Jose R. Zapata](https://joserzapata.github.io)

- [https://joserzapata.github.io/](https://joserzapata.github.io/)
- [https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/](https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/)